In [1]:
!pip install "wheel==0.34.2"

  Found existing installation: wheel 0.35.1
    Uninstalling wheel-0.35.1:
      Successfully uninstalled wheel-0.35.1


In [2]:
# Default Code
!pip3 install torch torchvision

# Code from Colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
torch.__version__

     |████████████████████████████████| 592.3MB 1.2MB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.


'0.3.0.post4'

In [4]:
# -*- coding:utf-8 -*-
"""
本实验基于类似class-oriented 所以效果要好很多
基于record-oriented未做，改一下数据集划分便可以。
"""
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pickle
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

TRAIN = False  # 训练标志
CONTINUE_TRAIN = False  # 接着上次某一次训练结果继续训练
TEST = False  # 测试标志 设置成True时候，需要指定加载哪个模型
PAPER_TEST = True  # 得到写paper使用的测试指标
SAVE_TEST_FIG = True
EPOCHS = 100
BATCH_SIZE = 32
Seqlength = 300
NUM_SEGS_CLASS = 5

qtdb_pkl = '/content/drive/MyDrive/2020-2 Deep Learning/팀프로젝트/[LAST] ECG-Segment-LSTM-master/qtdb_pkl/'  # 数据预处理后的路径，便于调试网络
save_path = '/content/drive/MyDrive/2020-2 Deep Learning/팀프로젝트/[LAST] ECG-Segment-LSTM-master/ckpt/'  # 保存模型的路径


if not os.path.exists(save_path):
    os.mkdir(save_path)


class ECGDataset(Dataset):
    """ecg dataset.
       返回字典：{'signal':  ,'label': }
    """
    def __init__(self, qtdb_pkl, data):
        """
        :param qtdb_pkl: 数据库存放路径
        :param data: 训练集和验证集数据
        """
        pkl = os.path.join(qtdb_pkl, data)
        with open(pkl, 'rb') as f:
            x, y = pickle.load(f)
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        signal = torch.from_numpy(self.x[idx]).float()
        label = torch.from_numpy(self.y[idx]).float()
        sample = {'signal': signal, 'label': label}
        return sample


class SegModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.features = torch.nn.Sequential(
            # torch.nn.Linear(in_features=input_size, out_features=hidden_size),
            torch.nn.LSTM(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True,
                          bidirectional=True),
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(2*hidden_size, 2*hidden_size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),

            torch.nn.Linear(2 * hidden_size, 2 * hidden_size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
        )
        self.output = torch.nn.Linear(2*hidden_size, out_size)

    def forward(self, x):
        """
        :param x: shape(batch, seq_len, input_size)
        :return:
        """
        batch, seq_len, nums_fea = x.size()
        features, _ = self.features(x)
        output = self.classifier(features)
        output = self.output(output.view(batch * seq_len, -1))
        return output


def train(net, data_loader, epochs):
    for step in range(epochs):
        net.train()
        for i, samples_batch in enumerate(data_loader):
            total = 0.0
            correct = 0.0

            output = net(samples_batch['signal'])
            target = samples_batch['label'].contiguous().view(-1).long()
            loss = criterion(output, target)

            _, predicted = torch.max(output.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()

            if (i+1) % 20 == 0:
                print("EPOCHS:{},Iter:{},Loss:{:.4f},Acc:{:.4f}".format(step, i+1, loss.item(), correct/total))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # 每2个epoch,保存一次模型
        if (step+1) % 2 == 0:
            torch.save(net, save_path+'epoch_{}.ckpt'.format(step))
        test(ecg_train_dl, 'train', step)
        test(ecg_val_dl, 'val', step)


def test(data_loader, str1, step):
    with torch.no_grad():
        right = 0.0
        total = 0.0
        net.eval()
        for sample in data_loader:
            output = net(sample['signal'])
            _, predicted = torch.max(output.data, 1)
            label = sample['label'].contiguous().view(-1).long()
            total += label.size(0)
            right += (predicted == label).sum().item()
        print("epoch:{},{} ACC: {:.4f}".format(step, str1, right / total))


def restore_net(ckpt):
    # load models
    with open(ckpt, 'rb') as f:
        net = torch.load(f)
    return net


def get_charateristic(y):
    Ppos = Qpos = Rpos =Spos = Tpos = 0
    for i, val in enumerate(y):
        if val == 1 and y[i-1] == 0:
            Ppos = i
        if val == 2 and y[i-1] == 0:
            Qpos = i
        if val == 2 and y[i+1] == 3:
            Rpos = i
        if val == 3 and y[i+1] == 0:
            Spos = i
        if val == 4 and y[i-1] == 0:
            Tpos = i
    return Ppos, Qpos, Rpos, Spos, Tpos


def point_equal(label, predict, tolerte):
    if predict <= label + tolerte * 250 and predict >= label- tolerte * 250:
        return True
    else:
        return False


def right_point(label_tuple, predict_tuple, tolerte):
    n = np.array([0, 0, 0, 0, 0])
    for i, (x, x_p) in enumerate(zip(label_tuple, predict_tuple)):
        if point_equal(x, x_p, tolerte):
            n[i] = 1
    return n


def plotlabel(y, bias):
    cmap = ['k', 'r', 'g', 'b', 'c', 'y']
    start = end = 0
    for i in range(len(y) - 1):
        if y[i] != y[i + 1]:
            end = i
            plt.plot(np.arange(start, end), y[start:end] - bias, cmap[int(y[i])])
            start = i + 1
        if i == len(y) - 2:
            end = len(y) - 1
            plt.plot(np.arange(start, end), y[start:end] - bias, cmap[int(y[i])])


def caculate_error(label_tuple, predict_tuple):
    error = np.zeros((5,))
    for i, (x, x_p) in enumerate(zip(label_tuple, predict_tuple)):
        error[i] = (x - x_p)/250*100  # (ms)
    return error



In [11]:
# loading data
ecg_train_db = ECGDataset(qtdb_pkl, 'train_data.pkl')
ecg_train_dl = DataLoader(ecg_train_db, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=1)

ecg_val_db = ECGDataset(qtdb_pkl, 'val_data.pkl')
ecg_val_dl = DataLoader(ecg_val_db, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=1)

if TRAIN:
    if CONTINUE_TRAIN:
        # continue training
        net = restore_net(save_path + 'epoch_102.ckpt')
    else:
        # model
        net = SegModel(input_size=2, hidden_size=32, num_layers=2, out_size=NUM_SEGS_CLASS)

    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    optimizer.zero_grad()

    train(net, ecg_train_dl, EPOCHS)

if TEST:
    # vis
    net = restore_net(save_path+'epoch_99.ckpt')
    net.eval()
    # test(ecg_val_dl, 'val', 4)
    for i, idx in enumerate([20, 60,
                              160, 280]):
        sample = ecg_val_db[idx]
        signal = sample['signal'].numpy()
        label = sample['label'].numpy()
        # plotecg(signal, label, 0, 1300)
        output = net(sample['signal'].unsqueeze(0))
        _, predict = torch.max(output, 1)
        # 将predict 和 label画出来
        predict = predict.numpy()
        x = np.arange(len(predict))
        plt.subplot(2, 2, i+1)
        plt.plot(x, signal[:, 0])
        plotlabel(label, 0.2)
        plotlabel(predict, 0.4)
    plt.show()

if PAPER_TEST:
    net = restore_net(save_path + 'epoch_99.ckpt')
    net.eval()
    print('waiting several minutes')
    right_point_num = np.array([0, 0, 0, 0, 0])
    error_array = np.zeros(shape=(len(ecg_val_db), 5))
    if SAVE_TEST_FIG:
        with PdfPages('test.pdf') as pdf:
            for i in range(len(ecg_val_db)):
                sample = ecg_val_db[i]
                signal = sample['signal'].numpy()
                label = sample['label'].numpy()
                # 得到预测结果
                output = net(sample['signal'].unsqueeze(0))
                _, predict = torch.max(output, 1)
                predict = predict.numpy()

                x = np.arange(Seqlength)
                plt.plot(x, signal[:, 0])
                plotlabel(label, 0.2)
                plotlabel(predict, 0.4)
                pdf.savefig()
                plt.close()

                label_points = get_charateristic(label)
                predict_points = get_charateristic(predict)

                error_array[i] = caculate_error(label_points, predict_points)

                # 得到p-end, QRS onset end , T-middle
                right_point_num += right_point(label_points,
                                                predict_points, 0.016)
    else:
        for i in range(len(ecg_val_db)):
            sample = ecg_val_db[i]
            signal = sample['signal'].numpy()
            label = sample['label'].numpy()
            # 得到预测结果
            output = net(sample['signal'].unsqueeze(0))
            _, predict = torch.max(output, 1)
            predict = predict.numpy()
            # 得到p-end, QRS onset end , T-middle
            right_point_num += right_point(get_charateristic(label),
                                            get_charateristic(predict), 0.025)
            
means = np.mean(error_array, axis=0)
SD = np.std(error_array, axis=0)
print(means)
print(SD)
print(right_point_num/len(ecg_val_db))

TypeError: ignored

In [7]:
!pip install wfdb

     |████████████████████████████████| 122kB 12.7MB/s 
     |████████████████████████████████| 6.8MB 3.3MB/s 
     |████████████████████████████████| 163kB 52.9MB/s 
  Created wheel for wfdb: filename=wfdb-3.1.1-cp36-none-any.whl size=117829 sha256=02afc4c28ae8d4daaf8f1f62ce7b5a6ad801e26453708b82c1377598b7dc1975
  Stored in directory: /root/.cache/pip/wheels/bc/d0/c1/90538d266ccba2d1076fbc9970192c7ea1a09c99df3e65c69b
Successfully built wfdb


In [8]:
#####################
### qtdatabase.py ###
#####################

import os
import math
import wfdb
import pickle
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from scipy.signal import medfilt
from matplotlib.backends.backend_pdf import PdfPages

In [10]:
Z_SCORE = True  # 对训练数据是否进行zscore归一化
SAVE_FIG = True  # 存储训练信号图像100个
train_percentage = 0.7  # 训练数据比例
features = 2  # 特征数目，用一条导联就是1，用两条导联就是2
Seqlength = 300  # 按照300个采样点
qtdbpath = './qtdb/'  # 数据路径
ann_suffix = 'q1c'  # 标注文件的后缀
qtdb_pickle_save = '/content/drive/MyDrive/2020-2 Deep Learning/팀프로젝트/[LAST] ECG-Segment-LSTM-master/qtdb_pkl/'  # 经过处理后保存数据路径

if not os.path.exists(qtdb_pickle_save):
    os.mkdir(qtdb_pickle_save)

# 下面几个文件没有P波，不参与本次实验
exclude = set()
exclude.update(["sel35", "sel36", "sel37", "sel50",
                "sel102", "sel104", "sel221",
                "sel232", "sel310"])
# 过滤不参与本次实验的record
datafiles = [x[:-4] for x in os.listdir(qtdbpath) if x[-4:] == '.dat']
record_names = list(set(datafiles)-exclude)


def baseline_correction(signals):
    """
    使用2个中值滤波器获得baseline,这个由于数据点数比较多，处理可能稍微慢一点
    中值滤波器的窗口分别为50=250*0.2s
                      150=250*0.6s
                      中值滤波需要奇数 所以是49 149
    :param signals：采样点信号 shape为(N,)
                   输入单导联体信号
    :return:去除基线后的采样信号
    """
    base_line = medfilt(signals, 49)
    base_line = medfilt(base_line, 149)
    signals = signals - base_line
    return signals


def ecg_preprocess(record):
    """
    过滤噪声的滤波器会对信号造成一定损失，为了保证数据的完整性，或者说
    检验网络的鲁棒性，我们不做噪声去除。
    :param record:
    :return:
    """
    record[:, 0] = baseline_correction(record[:, 0])
    record[:, 1] = baseline_correction(record[:, 1])
    return record


def remove_seq_gaps(x, y):
    """
    去掉非正常标注的片段， 如何衔接需要优化
    :param x:
    :param y:
    :return:
    """
    window = 150
    c = 0
    include = []
    print("filterering.")
    print("before shape x,y", x.shape, y.shape)
    for i in range(y.shape[0]):
        # 将连续未标注的超过150个点的整个未标注的去掉
        if 0 < c < window and y[i] != 0:
            for t in reversed(range(c)):
                include.append(i-t-1)
            c = 0
            include.append(i)
        elif c >= window and y[i] != 0:
            include.append(i)
            c = 0
        elif y[i] == 0:
            c += 1
        else:
            include.append(i)
    x, y = x[include, :], y[include]
    print(" after shape x,y", x.shape, y.shape)
    return x, y


def calculate_interv(poses):
    pt_interv = 10000
    pt_len = 0
    pp_interv = 10000
    for i, pose in enumerate(poses):
        if i < len(poses)-1:
            pt_interv = min(pt_interv, poses[i+1][0]-pose[-1])
            pp_interv = min(pp_interv, poses[i+1][0]-pose[0])
        pt_len = max(pt_len, pose[-1]-pose[0])
    return pt_interv, pt_len, pp_interv


def splitseg_single_beat(signal, label, poses):
    xx = np.zeros((len(poses), Seqlength, signal.shape[1]))
    yy = np.zeros((len(poses), Seqlength))
    for i, pose in enumerate(poses):
        x = np.zeros((Seqlength, features))
        y = np.zeros((Seqlength,))
        pstart, tend = pose[0], pose[-1]
        len_beat = tend-pstart+1
        start = (Seqlength-len_beat)//2
        x[start:start+len_beat] = signal[pstart:tend+1, :]
        y[start:start+len_beat] = label[pstart:tend+1]
        xx[i] = x
        yy[i] = y
    return xx, yy


def splitseg(signal, label, num, overlap):
    """
    创建LSTM训练和验证使用的片段，长度为num+2*overlap
    :param signal:
    :param label:
    :param num:
    :param overlap:
    :return:
    """
    length = signal.shape[0]
    num_seg = math.ceil(length / num)  # 计算可以得到多少个数据片段, 向上取整可能不是很合适，原因见下面的shape检查
    upper = num_seg * num  # math.ceil(8.5)=9
    print("splitting on", num, "with overlap of ", overlap, "total datapoints:", signal.shape[0], "; upper:", upper)
    xx = np.empty((num_seg, num + 2 * overlap, signal.shape[1]))  # 训练数据
    yy = np.empty((num_seg, num + 2 * overlap, ))  # 标签
    # 第一个片段取前num+overlap个 然后再在前面补overlap个零
    # 最后一个片段取后面num+overlap个 然后再在后面补overlap个零
    for i in range(num_seg):
        if i == 0:
            tmp = np.zeros((num+2*overlap, signal.shape[1]))
            tmp[overlap:, :] = signal[:num+overlap, :]
        elif i == num_seg-1:
            tmp = np.zeros((num+2*overlap, signal.shape[1]))
            tmp[:num+overlap, :] = signal[-(num+overlap):, :]
        else:
            # shape 检查，如果小于(num+2overlap,),则后面补零,这种情况会出现在7089扩充到8000
            tmp = np.zeros((num + 2 * overlap, signal.shape[1]))
            signal_i = signal[i*num-overlap: ((i+1)*num+overlap), :]
            tmp[:signal_i.shape[0]] = signal_i
        xx[i] = tmp

    for i in range(num_seg):
        if i == 0:
            tmp = np.zeros((num+2*overlap, ))
            tmp[overlap:] = label[:num+overlap]
        elif i == num_seg-1:
            tmp = np.zeros((num+2*overlap, ))
            tmp[:num+overlap] = label[-(num+overlap):]
        else:
            # shape 检查，如果小于(num+2overlap,),则后面补零,这种情况会出现在7089扩充到8000
            tmp = np.zeros((num + 2 * overlap, ))
            label_i = label[i*num-overlap: ((i+1)*num+overlap)]
            tmp[:label_i.shape[0]] = label_i
        yy[i] = tmp
    return xx, yy


def plotecg(x, y, start, end):
    x = x[start:end, 0]  # 只取第一条信号
    y = y[start:end]
    cmap = ['k', 'r', 'g', 'b']
    start = end = 0
    for i in range(len(y)-1):
        if y[i] != y[i+1]:
            end = i
            plt.plot(np.arange(start, end+1), x[start:end+1], cmap[int(y[i])])
            start = i+1
    plt.show()


def plotlabel(y, bias):
    cmap = ['k', 'r', 'g', 'b', 'c', 'y']
    start = end = 0
    for i in range(len(y) - 1):
        if y[i] != y[i + 1]:
            end = i
            plt.plot(np.arange(start, end), y[start:end]-bias, cmap[int(y[i])])
            start = i + 1
        if i == len(y)-2:
            end = len(y)-1
            plt.plot(np.arange(start, end), y[start:end] - bias, cmap[int(y[i])])


x = np.zeros((1, Seqlength, features))
y = np.zeros((1, Seqlength,))

min_tp = min_pp = 10000
max_len = 0

for record_name in record_names:
    # 先读标注文件，再根据标注文件的长度来读record
    annotation = wfdb.rdann(qtdbpath+record_name, extension=ann_suffix)
    start = annotation.sample[0]
    end = annotation.sample[-1]
    print('record {} start,end: {}, {}'.format(record_name, start, end))
    record, _ = wfdb.rdsamp(qtdbpath+record_name, sampfrom=start, sampto=end+1)

    # 两个信号都当做特征，所以每一个采样点2个特征
    signal = ecg_preprocess(record)

    # 将x进行zscore归一化
    if Z_SCORE:
        for i in range(signal.shape[1]):
            signal[:, i] = st.zscore(signal[:, i])

    Ann = list(zip(annotation.sample, annotation.symbol))
    poses = []
    for i in range(len(Ann)):
        ann = Ann[i]
        # 先找到P波,根据p波查找整个波形
        if ann[1] == 'p':
            pstart = ppos = pend = qpos = rpos = spos = tpos = tend = 0
            # 确定p波的起始和结束位置
            ppos = Ann[i][0]  # p波点
            if Ann[i - 1][1] == '(':
                pstart = Ann[i - 1][0]
            if Ann[i + 1][1] == ')':
                pend = Ann[i + 1][0]
            # p波紧随其后的就是QRS， 确定QRS波的位置
            if Ann[i + 3][1] == 'N':
                rpos = Ann[i + 3][0]
                if Ann[i + 2][1] == '(':
                    qpos = Ann[i + 2][0]
                if Ann[i + 4][1] == ')':
                    spos = Ann[i + 4][0]
                # 确认t波，因为有的没有‘(’,分情况讨论
                # 为了标注统一，只用t - ')'的信息,半个t wave
                if Ann[i + 6][1] == 't':
                    if Ann[i + 5][1] == '(':
                        # tpos = Ann[i + 5][0]
                        tpos = Ann[i + 6][0]
                    if Ann[i + 7][1] == ')':
                        tend = Ann[i + 7][0]
                elif Ann[i + 5][1] == 't':
                    tpos = Ann[i + 5][0]
                    if Ann[i + 6][1] == ')':
                        tend = Ann[i + 6][0]
                else:
                    print("can't find t wave")
            poses.append((pstart - start, ppos - start, pend - start, qpos - start,
                          rpos - start, spos - start, tpos - start, tend - start))
    label = np.zeros((end - start + 1))
    for pose in poses:
        (pstart, ppos, pend, qpos, rpos, spos, tpos, tend) = pose
        label[ppos: pend] = 1  # half P Wave
        label[qpos: rpos] = 2  # QR
        label[rpos: spos] = 3  # RS
        label[tpos: tend] = 4  # half t Wave

    # 计算相邻两个片段的min(前一个波的tend与后一个波的pstart的距离)
    # 计算一个片段的最大pt长度
    # 计算相邻两个片段的min(前一个波的pstart与后一个波的pstart的距离)
    # 这3个信息将用于决定，我们如何分割片段。
    # 该代码只使用一次。
    # 我们得到min_tp:1, max_len:283, min_pp:113 ，可以发现最小的tp间隔只有 1
    # 因为标注的片段是分散的，为了避免引入未标注数据，决定仅仅划分标注的片段。
    # 固定长度为300， 不到该长度的前后补零
    # pt_interv, pt_len, pp_interv = calculate_interv(poses)
    # min_tp = min(min_tp, pt_interv)
    # max_len = max(max_len, pt_len)
    # min_pp = min(min_pp, pp_interv)

    # 将过滤前后的信号与标注图画出来
    # plotecg(signal, label, 0, 500)
    # signal, label = remove_seq_gaps(signal, label)
    # plotecg(signal, label, 0, len(label))
    xx, yy = splitseg_single_beat(signal, label, poses)
    # xx, yy = splitseg(signal, label, 1000, 150)
    x = np.vstack((x, xx))
    y = np.vstack((y, yy))

print("min_tp:{}, max_len:{}, min_pp:{}".format(min_tp, max_len, min_pp))

# 将初始化的第一个sample去掉, 然后将片段打乱
x, y = x[1:], y[1:]
assert len(x) == len(y)
p = np.random.permutation(range(len(x)))
x, y = x[p], y[p]

# 划分训练集和验证集，然后保存下
nums = len(x)
train_len = int(math.ceil(nums*train_percentage))
x_train, y_train = x[:train_len], y[:train_len]
x_val, y_val = x[train_len:], y[train_len:]

print('训练集共有{}个片段，验证集共有{}个片段'.format(train_len, nums-train_len))

if SAVE_FIG:
    with PdfPages(qtdb_pickle_save+'example.pdf') as pdf:
        for i in range(100):
            signal = x_train[i][:, 0]
            x = np.arange(Seqlength)
            plt.plot(x, signal)
            plotlabel(y_train[i], 0.2)
            pdf.savefig()
            plt.close()

with open(qtdb_pickle_save+'train_data.pkl', 'wb') as f:
    pickle.dump((x_train, y_train), f)

with open(qtdb_pickle_save+'val_data.pkl', 'wb') as f:
    pickle.dump((x_val, y_val), f)

FileNotFoundError: ignored